# comercio_ext_indices.tb_referencia_ncm_cuci
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CUCI_delta`
## 📌 Descrição do arquivo

Classificação **CUCI** hierárquica (Seção → Divisão → Grupo → Subgrupo → Item).

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_CUCI_ITEM`|Código do item CUCI|Nível mais detalhado|
|`NO_CUCI_ITEM`|Nome do item|Descrição do item|
|`CO_CUCI_SUB`|Código do subgrupo|Nível acima do item|
|`NO_CUCI_SUB`|Nome do subgrupo|Descrição do subgrupo|
|`CO_CUCI_GRUPO`|Código do grupo|Agrupamento intermediário|
|`NO_CUCI_GRUPO`|Nome do grupo|Descrição do grupo|
|`CO_CUCI_DIVISAO`|Código da divisão|Agrupamento amplo|
|`NO_CUCI_DIVISAO`|Nome da divisão|Descrição da divisão|
|`CO_CUCI_SEC`|Código da seção|Macrogrupo CUCI|
|`NO_CUCI_SEC`|Nome da seção|Macrogrupo|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CUCI_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_cuci/"   
val silverTable =    "tb_referencia_ncm_cuci"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_CUCI_ITEM" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_ITEM" , StringType ,  nullable = false ) ,
      StructField ( "CO_CUCI_SUB" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_SUB" , StringType ,  nullable = false ) ,
      StructField ( "CO_CUCI_GRUPO" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_GRUPO" , StringType ,  nullable = false ) ,
      StructField ( "CO_CUCI_DIVISAO" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_DIVISAO" , StringType ,  nullable = false ) ,
      StructField ( "CO_CUCI_SEC" , StringType ,  nullable = false ) ,
      StructField ( "NO_CUCI_SEC" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_CUCI_ITEM", trim(upper(col("CO_CUCI_ITEM").cast(StringType))))
  .withColumn("NO_CUCI_ITEM", col("NO_CUCI_ITEM").cast(StringType))
  .withColumn("CO_CUCI_SUB", trim(upper(col("CO_CUCI_SUB").cast(StringType))))
  .withColumn("NO_CUCI_SUB", col("NO_CUCI_SUB").cast(StringType))
  .withColumn("CO_CUCI_GRUPO", trim(upper(col("CO_CUCI_GRUPO").cast(StringType))))
  .withColumn("NO_CUCI_GRUPO", col("NO_CUCI_GRUPO").cast(StringType))
  .withColumn("CO_CUCI_DIVISAO", trim(upper(col("CO_CUCI_DIVISAO").cast(StringType))))
  .withColumn("NO_CUCI_DIVISAO", col("NO_CUCI_DIVISAO").cast(StringType))
  .withColumn("CO_CUCI_SEC", trim(upper(col("CO_CUCI_SEC").cast(StringType))))
  .withColumn("NO_CUCI_SEC", col("NO_CUCI_SEC").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NCM_CUCI_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_CUCI_ITEM").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_CUCI_ITEM"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_CUCI_ITEM"), col("NO_CUCI_ITEM"), col("CO_CUCI_SUB"), col("NO_CUCI_SUB"), col("CO_CUCI_GRUPO"), col("NO_CUCI_GRUPO"), col("CO_CUCI_DIVISAO"), col("NO_CUCI_DIVISAO"), col("CO_CUCI_SEC"), col("NO_CUCI_SEC"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}